In [64]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
from statsmodels.iolib import load_pickle
from linearmodels.panel import RandomEffects

In [55]:
cre = load_pickle("model/ind_clust_entity_cre.pickle")
print(cre.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:               LEVERAGE   R-squared:                        0.2733
Estimator:              RandomEffects   R-squared (Between):              0.4508
No. Observations:                1728   R-squared (Within):               0.2401
Date:                Mon, Apr 01 2024   R-squared (Overall):              0.4129
Time:                        13:56:57   Log-likelihood                    1505.4
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      53.744
Entities:                         216   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(12,1715)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             36.989
                            

In [56]:
sgx = pd.read_csv("data/clean_sgx.csv")
covid_sgx = pd.read_csv("data/covid_clean_sgx.csv")
normal_sgx = pd.read_csv("data/normal_clean_sgx.csv")

## COVID impact

In [57]:
sgx = sgx.set_index(['Company Code', 'Year'], drop= False)
sgx['COVID'] = pd.Categorical(['covid' if year > 2019 else 'normal' for year in sgx['Year']], categories= ['normal', 'covid'])
sgx['Company Code'] = pd.Categorical(sgx['Company Code'])

sgx

Company  NACE    Year            TA  \
Company Code Year                                                            
3            2015.0  A SONIC AEROSPACE LIMITED    26  2015.0  6.107200e+07   
             2016.0  A SONIC AEROSPACE LIMITED    26  2016.0  6.467500e+07   
             2017.0  A SONIC AEROSPACE LIMITED    26  2017.0  7.524600e+07   
             2018.0  A SONIC AEROSPACE LIMITED    26  2018.0  7.359400e+07   
             2019.0  A SONIC AEROSPACE LIMITED    26  2019.0  6.428400e+07   
...                                        ...   ...     ...           ...   
573          2018.0   YONGNAM HOLDINGS LIMITED    71  2018.0  3.503341e+08   
             2019.0   YONGNAM HOLDINGS LIMITED    71  2019.0  3.429246e+08   
             2020.0   YONGNAM HOLDINGS LIMITED    71  2020.0  2.913047e+08   
             2021.0   YONGNAM HOLDINGS LIMITED    71  2021.0  2.595798e+08   
             2022.0   YONGNAM HOLDINGS LIMITED    71  2022.0  2.308791e+08   

                               CA            FA            TL            CL  \
Company Code Year                                                             
3            2015.0  4.941900e+07  1.079300e+07  3.331800e+07  3.160000e+07   
             2016.0  5.772600e+07  6.258000e+06  3.964200e+07  3.835100e+07   
             2017.0  6.803700e+07  6.118000e+06  5.161200e+07  5.066000e+07   
             2018.0  6.584300e+07  5.849000e+06  4.928100e+07  4.870300e+07   
             2019.0  5.679200e+07  7.293000e+06  3.731000e+07  3.643600e+07   
...                           ...           ...           ...           ...   
573          2018.0  1.196307e+08  2.306594e+08  1.703341e+08  1.032232e+08   
             2019.0  1.037248e+08  2.351062e+08  2.013324e+08  1.504135e+08   
             2020.0  6.392860e+07  2.236608e+08  2.061644e+08  1.618667e+08   
             2021.0  6.673153e+07  1.896449e+08  1.906858e+08  1.146763e+08   
             2022.0  6.542243e+07  1.623702e+08  1.868504e+08  1.612576e+08   

                             DEBT         TAXEX  ...  PROFITABILITY      TANG  \
Company Code Year                                ...                            
3            2015.0 -1.437900e+07  2.910000e+05  ...      -0.094659  0.176726   
             2016.0 -1.338600e+07 -2.650000e+05  ...      -0.010839  0.096761   
             2017.0 -1.792500e+07 -2.160000e+05  ...      -0.013888  0.081307   
             2018.0 -1.906200e+07 -1.820000e+05  ...       0.013371  0.079477   
             2019.0 -1.432700e+07 -3.740000e+05  ...       0.078838  0.113450   
...                           ...           ...  ...            ...       ...   
573          2018.0  9.217834e+07  8.678195e+06  ...      -0.059740  0.658398   
             2019.0  1.227331e+08  3.608225e+06  ...      -0.033475  0.685591   
             2020.0  1.292187e+08 -4.172907e+06  ...      -0.064894  0.767790   
             2021.0  1.165392e+08  4.512836e+04  ...       0.027514  0.730584   
             2022.0  1.057712e+08 -9.296445e+04  ...       0.009406  0.703270   

                           RISK    LIQUID  NDTSHIELD   TAXRATE          FCFF  \
Company Code Year                                                              
3            2015.0  -82.603240  1.563892   0.041296 -0.029991  4.729897e+06   
             2016.0   12.036318  1.505202   0.012911  0.097070  6.670989e+05   
             2017.0    1.605162  1.343012   0.010791  0.150313  4.998374e+06   
             2018.0  -20.179642  1.351929   0.010245 -0.140108  8.313457e+05   
             2019.0 -193.544766  1.558678   0.029790 -0.173228  3.991890e+05   
...                         ...       ...        ...       ...           ...   
573          2018.0   -3.777753  1.158952   0.062545 -0.302336 -1.035043e+07   
             2019.0   -0.690731  0.689598   0.071469 -0.099419  4.195210e+07   
             2020.0   -0.541603  0.394946   0.093303  0.066056  5.687663e+07   
             2021.0   -2.513608  0.581912   0.078202 -0.002244

In [58]:
endo_var = 'LEVERAGE'
exog_vars = ['SIZE',
             'PROFITABILITY',
             'TANG',
             'LIQUID',
             'MCAP',
             'SOLV']
dummy_var = 'COVID'

endo = sgx[endo_var]

mean_exog_vars = ['avg' + var for var in exog_vars]
sgx[mean_exog_vars] = sgx[exog_vars].groupby(level= 'Company Code').transform('mean')
exog = sm.add_constant(sgx[exog_vars + mean_exog_vars + [dummy_var]])
sgx.drop(columns= mean_exog_vars)

covid_cre_mod = RandomEffects(endo, exog)

covid_cre_fit = covid_cre_mod.fit(cov_type= 'clustered', cluster_entity= True)

print(covid_cre_fit.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:               LEVERAGE   R-squared:                        0.2733
Estimator:              RandomEffects   R-squared (Between):              0.4508
No. Observations:                1728   R-squared (Within):               0.2402
Date:                Tue, Apr 02 2024   R-squared (Overall):              0.4130
Time:                        03:36:46   Log-likelihood                    1506.0
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      49.576
Entities:                         216   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(13,1714)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             34.214
                            

In [59]:
covid_sgx = covid_sgx.set_index(['Company Code', 'Year'])
covid_sgx

Company  NACE            TA  \
Company Code Year                                                    
3            2020.0  A SONIC AEROSPACE LIMITED    26  7.897600e+07   
             2021.0  A SONIC AEROSPACE LIMITED    26  1.069980e+08   
             2022.0  A SONIC AEROSPACE LIMITED    26  9.621900e+07   
10           2020.0               ACMA LIMITED    46  4.271160e+07   
             2021.0               ACMA LIMITED    46  3.307687e+07   
...                                        ...   ...           ...   
572          2021.0   YONGMAO HOLDINGS LIMITED    28  2.928690e+08   
             2022.0   YONGMAO HOLDINGS LIMITED    28  2.961636e+08   
573          2020.0   YONGNAM HOLDINGS LIMITED    71  2.913047e+08   
             2021.0   YONGNAM HOLDINGS LIMITED    71  2.595798e+08   
             2022.0   YONGNAM HOLDINGS LIMITED    71  2.308791e+08   

                               CA            FA            TL            CL  \
Company Code Year                                                             
3            2020.0  7.265200e+07  6.104000e+06  4.428600e+07  4.393800e+07   
             2021.0  1.002060e+08  6.489000e+06  6.376400e+07  6.330200e+07   
             2022.0  8.911300e+07  6.150000e+06  5.022600e+07  4.882200e+07   
10           2020.0  3.046971e+07  1.211709e+07  3.411013e+07  2.856289e+07   
             2021.0  2.543168e+07  7.594141e+06  2.852038e+07  2.428276e+07   
...                           ...           ...           ...           ...   
572          2021.0  1.916742e+08  8.304555e+07  1.525279e+08  1.416881e+08   
             2022.0  1.916892e+08  8.361025e+07  1.522947e+08  1.420764e+08   
573          2020.0  6.392860e+07  2.236608e+08  2.061644e+08  1.618667e+08   
             2021.0  6.673153e+07  1.896449e+08  1.906858e+08  1.146763e+08   
             2022.0  6.542243e+07  1.623702e+08  1.868504e+08  1.612576e+08   

                             DEBT         TAXEX         INTEX  ...       SIZE  \
Company Code Year                                              ...              
3            2020.0 -2.790800e+07 -2.820000e+05  2.165000e+06  ...  18.184655   
             2021.0 -3.795500e+07 -1.550000e+06  5.991000e+06  ...  18.488321   
             2022.0 -4.558800e+07 -7.910000e+05  1.640000e+06  ...  18.382137   
10           2020.0  1.281673e+07 -3.645715e+05  2.117086e+06  ...  17.569981   
             2021.0  7.668862e+06 -1.657173e+05 -1.495154e+06  ...  17.314345   
...                           ...           ...           ...  ...        ...   
572          2021.0  1.768001e+07 -1.987009e+06  8.435997e+06  ...  19.495236   
             2022.0  1.358404e+07 -1.307855e+06  6.112980e+06  ...  19.506423   
573          2020.0  1.292187e+08 -4.172907e+06  1.708872e+07  ...  19.489881   
             2021.0  1.165392e+08  4.512836e+04  6.949027e+06  ...  19.374575   
             2022.0  1.057712e+08 -9.296445e+04  1.471962e+07  ...  19.257405   

                     PROFITABILITY      TANG      RISK    LIQUID  NDTSHIELD  \
Company Code Year                                                             
3            2020.0       0.126747  0.077289  5.664351  1.653512   0.022792   
             2021.0       0.118694  0.060646  0.463645  1.582983   0.015346   
             2022.0       0.079766  0.063917  2.740224  1.825263   0.020245   
10           2020.0       0.006588  0.283695  2.053758  1.066758   0.075245   
             2021.0      -0.032096  0.229591 -9.226522  1.047314   0.088213   
...                            ...       ...       ...       ...        ...   
572          2021.0       0.058167  0.283559  0.032819  1.352790   0.006487   
             2022.0       0.047870  0.282311  0.797621  1.349197   0.006823   
573          2020.0      -0.064894  0.767790 -0.541603  0.394946   0.093303   
             2021.0       0.027514  0.730584 -2.513608  0.581912   0.078202   
             2022.0       0.009406  0.703270  2.244895  0.405701   0.082164   

                  

In [60]:
endo_var = 'LEVERAGE'
exog_vars = ['SIZE',
             'PROFITABILITY',
             'TANG',
             'LIQUID',
             'MCAP',
             'SOLV']

endo = covid_sgx[endo_var]

mean_exog_vars = ['avg' + var for var in exog_vars]
covid_sgx[mean_exog_vars] = covid_sgx[exog_vars].groupby(level= 'Company Code').transform('mean')
exog = sm.add_constant(covid_sgx[exog_vars + mean_exog_vars])
covid_sgx.drop(columns= mean_exog_vars)

covid_cre_mod = RandomEffects(endo, exog)

covid_cre_fit = covid_cre_mod.fit(cov_type= 'clustered', cluster_entity= True)

print(covid_cre_fit.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:               LEVERAGE   R-squared:                        0.3281
Estimator:              RandomEffects   R-squared (Between):              0.3921
No. Observations:                 648   R-squared (Within):               0.2920
Date:                Tue, Apr 02 2024   R-squared (Overall):              0.3857
Time:                        03:36:46   Log-likelihood                    825.47
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      25.835
Entities:                         216   P-value                           0.0000
Avg Obs:                       3.0000   Distribution:                  F(12,635)
Min Obs:                       3.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             19.211
                            

In [61]:
normal_sgx = normal_sgx.set_index(['Company Code', 'Year'])
normal_sgx

Company  NACE            TA  \
Company Code Year                                                    
3            2015.0  A SONIC AEROSPACE LIMITED    26  6.107200e+07   
             2016.0  A SONIC AEROSPACE LIMITED    26  6.467500e+07   
             2017.0  A SONIC AEROSPACE LIMITED    26  7.524600e+07   
             2018.0  A SONIC AEROSPACE LIMITED    26  7.359400e+07   
             2019.0  A SONIC AEROSPACE LIMITED    26  6.428400e+07   
...                                        ...   ...           ...   
573          2015.0   YONGNAM HOLDINGS LIMITED    71  3.821897e+08   
             2016.0   YONGNAM HOLDINGS LIMITED    71  3.659407e+08   
             2017.0   YONGNAM HOLDINGS LIMITED    71  3.757609e+08   
             2018.0   YONGNAM HOLDINGS LIMITED    71  3.503341e+08   
             2019.0   YONGNAM HOLDINGS LIMITED    71  3.429246e+08   

                               CA            FA            TL            CL  \
Company Code Year                                                             
3            2015.0  4.941900e+07  1.079300e+07  3.331800e+07  3.160000e+07   
             2016.0  5.772600e+07  6.258000e+06  3.964200e+07  3.835100e+07   
             2017.0  6.803700e+07  6.118000e+06  5.161200e+07  5.066000e+07   
             2018.0  6.584300e+07  5.849000e+06  4.928100e+07  4.870300e+07   
             2019.0  5.679200e+07  7.293000e+06  3.731000e+07  3.643600e+07   
...                           ...           ...           ...           ...   
573          2015.0  1.636325e+08  2.174199e+08  1.718686e+08  1.108282e+08   
             2016.0  1.476575e+08  2.172461e+08  1.591834e+08  1.113158e+08   
             2017.0  1.307519e+08  2.450090e+08  1.518113e+08  1.283181e+08   
             2018.0  1.196307e+08  2.306594e+08  1.703341e+08  1.032232e+08   
             2019.0  1.037248e+08  2.351062e+08  2.013324e+08  1.504135e+08   

                             DEBT         TAXEX         INTEX  ...       SIZE  \
Company Code Year                                              ...              
3            2015.0 -1.437900e+07  2.910000e+05  1.397000e+06  ...  17.927564   
             2016.0 -1.338600e+07 -2.650000e+05  1.194000e+06  ...  17.984885   
             2017.0 -1.792500e+07 -2.160000e+05 -3.950000e+05  ...  18.136273   
             2018.0 -1.906200e+07 -1.820000e+05 -1.061000e+06  ...  18.114074   
             2019.0 -1.432700e+07 -3.740000e+05  9.940000e+05  ...  17.978821   
...                           ...           ...           ...  ...        ...   
573          2015.0  9.499187e+07  1.020581e+06  2.229295e+06  ...  19.761428   
             2016.0  6.786421e+07  3.692180e+06 -4.315149e+06  ...  19.717982   
             2017.0  6.448975e+07  2.448751e+06 -4.901242e+06  ...  19.744464   
             2018.0  9.217834e+07  8.678195e+06 -1.413687e+07  ...  19.674398   
             2019.0  1.227331e+08  3.608225e+06  3.050772e+05  ...  19.653021   

                     PROFITABILITY      TANG        RISK    LIQUID  NDTSHIELD  \
Company Code Year                                                               
3            2015.0      -0.094659  0.176726  -82.603240  1.563892   0.041296   
             2016.0      -0.010839  0.096761   12.036318  1.505202   0.012911   
             2017.0      -0.013888  0.081307    1.605162  1.343012   0.010791   
             2018.0       0.013371  0.079477  -20.179642  1.351929   0.010245   
             2019.0       0.078838  0.113450 -193.544766  1.558678   0.029790   
...                            ...       ...         ...       ...        ...   
573          2015.0       0.053104  0.568880   -5.749879  1.476452   0.050781   
             2016.0      -0.009018  0.593665 -159.408562  1.326474   0.052460   
             2017.0       0.019051  0.652034  -10.040239  1.018967   0.059441   
             2018.0      -0.059740  0.658398   -3.777753  1.158952   0.062545   
             2019.0      -0.033475  0.685591   -0.690731  0.689598   0.071

In [62]:
endo_var = 'LEVERAGE'
exog_vars = ['SIZE',
             'PROFITABILITY',
             'TANG',
             'LIQUID',
             'MCAP',
             'SOLV']

endo = normal_sgx[endo_var]

mean_exog_vars = ['avg' + var for var in exog_vars]
normal_sgx[mean_exog_vars] = normal_sgx[exog_vars].groupby(level= 'Company Code').transform('mean')
exog = sm.add_constant(normal_sgx[exog_vars + mean_exog_vars])
normal_sgx.drop(columns= mean_exog_vars)

normal_cre_mod = RandomEffects(endo, exog)

normal_cre_fit = normal_cre_mod.fit(cov_type= 'clustered', cluster_entity= True)

print(normal_cre_fit.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:               LEVERAGE   R-squared:                        0.3146
Estimator:              RandomEffects   R-squared (Between):              0.4503
No. Observations:                1080   R-squared (Within):               0.2718
Date:                Tue, Apr 02 2024   R-squared (Overall):              0.4245
Time:                        03:36:46   Log-likelihood                    1045.1
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      40.812
Entities:                         216   P-value                           0.0000
Avg Obs:                       5.0000   Distribution:                 F(12,1067)
Min Obs:                       5.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             46.546
                            

## Sector Impact

In [67]:
nace_mapping_dict = {
    1: 'Agriculture', 2: 'Agriculture', 3: 'Agriculture', 
    5: 'Mining', 6: 'Mining', 7: 'Mining', 
    8: 'Mining', 9: 'Mining', 
    10: 'Manufacturing', 11: 'Manufacturing', 12: 'Manufacturing', 
    13: 'Manufacturing', 14: 'Manufacturing', 15: 'Manufacturing', 
    16: 'Manufacturing', 17: 'Manufacturing', 18: 'Manufacturing', 
    19: 'Manufacturing', 20: 'Manufacturing', 21: 'Manufacturing', 
    22: 'Manufacturing', 23: 'Manufacturing', 24: 'Manufacturing', 
    25: 'Manufacturing', 26: 'Manufacturing', 27: 'Manufacturing', 
    28: 'Manufacturing', 29: 'Manufacturing', 30: 'Manufacturing', 
    31: 'Manufacturing', 32: 'Manufacturing', 33: 'Manufacturing', 
    34: 'Energy', 35: 'Energy', 
    36: 'Utilities', 37: 'Utilities', 38: 'Utilities', 
    39: 'Utilities', 
    41: 'Construction', 42: 'Construction', 43: 'Construction', 
    45: 'Wholesale and Retail', 46: 'Wholesale and Retail', 
    47: 'Wholesale and Retail', 48: 'Wholesale and Retail', 
    49: 'Transport', 50: 'Transport', 51: 'Transport', 
    52: 'Transport', 53: 'Transport', 
    55: 'Service', 56: 'Service', 
    58: 'Infocomm', 59: 'Infocomm', 60: 'Infocomm', 
    61: 'Infocomm', 62: 'Infocomm', 63: 'Infocomm', 
    64: 'Finance', 65: 'Finance', 66: 'Finance', 
    68: 'Real Estate', 
    69: 'Professional Service', 70: 'Professional Service', 
    71: 'Professional Service', 72: 'Professional Service', 
    73: 'Professional Service', 74: 'Professional Service', 
    75: 'Professional Service', 
    77: 'Administrative Service', 78: 'Administrative Service', 
    79: 'Administrative Service', 80: 'Administrative Service', 
    81: 'Administrative Service', 82: 'Administrative Service', 
    84: 'Public Service', 
    85: 'Education', 
    86: 'Healthcare', 87: 'Healthcare', 88: 'Healthcare', 
    90: 'Entertainment', 91: 'Entertainment', 92: 'Entertainment',
    93: 'Entertainment', 
    94: 'Others', np.nan: 'Others', '': 'Others'
}

sgx['Sector'] = sgx['NACE'].replace(nace_mapping_dict)

In [119]:
for sector in sgx['Sector'].unique().tolist():
    try:
        print(f'\nCRE Estimation of {sector} firms:\n')
        sector_df = sgx[sgx['Sector'] == sector]

        endo_var = 'LEVERAGE'
        exog_vars = ['SIZE',
                    'PROFITABILITY',
                    'TANG',
                    'LIQUID',
                    'MCAP',
                    'SOLV']

        endo = sector_df[endo_var]

        mean_exog_vars = ['avg' + var for var in exog_vars]
        sector_df[mean_exog_vars] = sector_df[exog_vars].groupby(level= 'Company Code').transform('mean')
        exog = sm.add_constant(sector_df[exog_vars + mean_exog_vars])
        sector_df.drop(columns= mean_exog_vars)

        normal_cre_mod = RandomEffects(endo, exog)

        normal_cre_fit = normal_cre_mod.fit(cov_type= 'clustered', cluster_entity= True)

        print(normal_cre_fit.summary)
        del sector_df
    except:
        pass


CRE Estimation of Manufacturing firms:

                        RandomEffects Estimation Summary                        
Dep. Variable:               LEVERAGE   R-squared:                        0.3814
Estimator:              RandomEffects   R-squared (Between):              0.4957
No. Observations:                 720   R-squared (Within):               0.3634
Date:                Tue, Apr 02 2024   R-squared (Overall):              0.4727
Time:                        06:11:31   Log-likelihood                    698.59
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      36.324
Entities:                          90   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(12,707)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):       

C:\Users\yungq\AppData\Local\Temp\ipykernel_22464\3632006537.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sector_df[mean_exog_vars] = sector_df[exog_vars].groupby(level= 'Company Code').transform('mean')
C:\Users\yungq\AppData\Local\Temp\ipykernel_22464\3632006537.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sector_df[mean_exog_vars] = sector_df[exog_vars].groupby(level= 'Company Code').transform('mean')


                        RandomEffects Estimation Summary                        
Dep. Variable:               LEVERAGE   R-squared:                        0.3709
Estimator:              RandomEffects   R-squared (Between):              0.6485
No. Observations:                 176   R-squared (Within):               0.3377
Date:                Tue, Apr 02 2024   R-squared (Overall):              0.6130
Time:                        06:11:31   Log-likelihood                    182.13
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      8.0075
Entities:                          22   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(12,163)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             300.01
                            

C:\Users\yungq\AppData\Local\Temp\ipykernel_22464\3632006537.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sector_df[mean_exog_vars] = sector_df[exog_vars].groupby(level= 'Company Code').transform('mean')
C:\Users\yungq\AppData\Local\Temp\ipykernel_22464\3632006537.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sector_df[mean_exog_vars] = sector_df[exog_vars].groupby(level= 'Company Code').transform('mean')
C:\Users\yungq\AppData\Local\Temp\ipykernel_22464\3632006537.py:17: SettingWithCopyWarni

                        RandomEffects Estimation Summary                        
Dep. Variable:               LEVERAGE   R-squared:                        0.8623
Estimator:              RandomEffects   R-squared (Between):              0.9858
No. Observations:                  64   R-squared (Within):               0.3651
Date:                Tue, Apr 02 2024   R-squared (Overall):              0.8623
Time:                        06:11:31   Log-likelihood                    64.977
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      26.612
Entities:                           8   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                   F(12,51)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):          2.944e+14
                            

C:\Users\yungq\AppData\Local\Temp\ipykernel_22464\3632006537.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sector_df[mean_exog_vars] = sector_df[exog_vars].groupby(level= 'Company Code').transform('mean')
C:\Users\yungq\AppData\Local\Temp\ipykernel_22464\3632006537.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sector_df[mean_exog_vars] = sector_df[exog_vars].groupby(level= 'Company Code').transform('mean')
C:\Users\yungq\AppData\Local\Temp\ipykernel_22464\3632006537.py:17: SettingWithCopyWarni